In [11]:
import cv2 as cv
import numpy as np
import math

In [12]:
class Optical_Flow_lib():
    def __init__(self,istream):
        super(Optical_Flow_lib, self).__init__()
        self.istream=istream
    def Optical_Flow(self,start_frame_num):
        self.istream.set(cv.CAP_PROP_POS_FRAMES, start_frame_num)
        _, frame = self.istream.read()
        _, next_frame = self.istream.read() 
        
        frame = cv.resize(frame, (224, 224),interpolation=cv.INTER_CUBIC)
        next_frame = cv.resize(next_frame, (224, 224),interpolation=cv.INTER_CUBIC)
        
        gray=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
        next_gray=cv.cvtColor(next_frame,cv.COLOR_BGR2GRAY)
        
        flow = cv.calcOpticalFlowFarneback(gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0) 
        
        mag=np.sqrt(flow[...,0]**2+flow[...,1]**2)
        ang=np.arctan2(flow[...,1],flow[...,0])% (2 * np.pi)
            
        return frame,mag,ang
    @staticmethod 
    def visual_flow(magnitude,angle):
        mask = np.zeros((224,224,3),dtype=np.uint8)
        mask[..., 1]=255
        mask[..., 0] = angle.reshape(224,224) * 180 / np.pi / 2
        #mask[..., 0] = angle/2
        mask[..., 2] = cv.normalize(magnitude.reshape(224,224), None, 0, 255, cv.NORM_MINMAX) 

        Flow = cv.cvtColor(mask, cv.COLOR_HSV2BGR) 
        return Flow
    @staticmethod
    def calc_prob(magnitude,angle,bin_num=[8,8],max_num=[4,2*np.pi]):
        hist_2d=np.zeros((bin_num[0],bin_num[1]))
        for m,a in zip(magnitude.ravel(),angle.ravel()):
            x=m//(max_num[0]/bin_num[0])
            y=a//(max_num[1]/bin_num[1])
            x= bin_num[0]-1 if x>=bin_num[0]-1 else int(x)
            y= bin_num[1]-1 if y>=bin_num[1]-1 else int(y)
            hist_2d[x,y]+=1
        prob=hist_2d/(224*224)
        return prob
    @staticmethod
    def entropy(prob):
        h=[(-i*(np.log2(i))) for i in prob.ravel() if i !=0.0]
        return np.sum(h)
    @staticmethod
    def prob_and_entropy(magnitude,angle,bin_num=[8,8],max_num=[4,2*np.pi]):
        prob=Optical_Flow_lib.calc_prob(magnitude,angle,bin_num=bin_num,max_num=max_num)
        h=[(-i*(np.log2(i))) for i in prob.ravel() if i !=0.0]
        return np.sum(h)
    
        